In [1]:
import pandas as pd

from def_clean import DefensiveCleaning

def_cleaning = DefensiveCleaning(n_cuts=11)

..............................initializing
positional data already downloaded.
reading positional data.
returning positional data.
..data downloaded...


In [2]:
week = def_cleaning.combine_week(1)

...Week 1 loaded...
...filtered...
...transformed...
...time reduced...
...starting dataframe generated...
...action type generated...
.....Week 1 COMPLETE.....
